In [1]:
import json

from pathlib import Path

import pandas as pd
import numpy as np

import biopsykit as bp

import matplotlib.pyplot as plt

from pepbench.io import load_challenge_results_from_folder
from pepbench.io import convert_hz_to_ms
from pepbench.datasets import EmpkinsDataset, GuardianDataset

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [2]:
root_path = Path("../../")

In [3]:
result_path = root_path.joinpath("results")
result_path

WindowsPath('../../results')

In [4]:
deploy_type = "local"

config_dict = json.load(root_path.joinpath("config.json").open(encoding="utf-8"))

empkins_base_path = Path(config_dict[deploy_type]["empkins_path"])
guardian_base_path = Path(config_dict[deploy_type]["guardian_path"])
print(empkins_base_path)

\Users\sebas\Development\ResearchInternship\Data\2024_08_PEP_Benchmarking\EmpkinS_Dataset


In [5]:
save_results = True

In [6]:
dataset_empkins = EmpkinsDataset(empkins_base_path, use_cache=True, only_labeled=True)
fs_empkins = dataset_empkins.sampling_rate_icg
print(f"Sampling rate ICG: {fs_empkins}")
dataset_guardian = GuardianDataset(guardian_base_path, use_cache=True, only_labeled=True)
fs_guardian = dataset_guardian.sampling_rate_icg
print(f"Sampling rate ICG: {fs_guardian}")

Sampling rate ICG: 1000
Sampling rate ICG: 500


In [7]:
algo_levels = ["q_wave_algorithm", "b_point_algorithm", "outlier_correction_algorithm"]
algo_level_mapping = dict(zip(algo_levels, ["Q-Wave Algorithm", "B-Point Algorithm", "Outlier Correction"]))

In [8]:
results_empkins = load_challenge_results_from_folder(
    result_path.joinpath("empkins_dataset_reference_q_wave").resolve(), index_cols_per_sample=["participant", "condition", "phase"]
).per_sample
results_guardian = load_challenge_results_from_folder(
    result_path.joinpath("guardian_dataset_reference_q_wave").resolve(), index_cols_per_sample=["participant", "phase"]
).per_sample

In [9]:
results_empkins = results_empkins.droplevel("condition")

In [10]:
results_empkins_norm = results_empkins.copy()
results_guardian_norm = results_guardian.copy()
results_empkins_norm[("b_point_sample", "estimated")] = results_empkins[("b_point_sample", "estimated")] - results_empkins[("heartbeat_start_sample", "estimated")]
results_empkins_norm[("b_point_sample", "reference")] = results_empkins[("b_point_sample", "reference")] - results_empkins[("heartbeat_start_sample", "estimated")]
results_guardian_norm[("b_point_sample", "estimated")] = results_guardian[("b_point_sample", "estimated")] - results_guardian[("heartbeat_start_sample", "estimated")]
results_guardian_norm[("b_point_sample", "reference")] = results_guardian[("b_point_sample", "reference")] - results_guardian[("heartbeat_start_sample", "estimated")]

In [11]:
results_guardian_norm

heartbeat_id  \
                                                                                                         estimated   
q_wave_algorithm b_point_algorithm               outlier_correction_algorithm participant phase                      
q-wave-reference arbol2017-isoelectric-crossings forouzanfar2018              GDN0005     Pause    0             0   
                                                                                                   1             1   
                                                                                                   2             2   
                                                                                                   3             3   
                                                                                                   4             4   
...                                                                                                            ...   
                 stern1985                       none                         GDN0030     TiltDown 51           51   
                                                                                                   52           52   
                                                                                                   53           53   
                                                                                                   54           54   
                                                                                                   55           55   

                                                                                                                 \
                                                                                                      reference   
q_wave_algorithm b_point_algorithm               outlier_correction_algorithm participant phase                   
q-wave-reference arbol2017-isoelectric-crossings forouzanfar2018              GDN0005     Pause    0          0   
                                                                                                   1          1   
                                                                                                   2          2   
                                                                                                   3          3   
                                                                                                   4          4   
...                                                                                                         ...   
                 stern1985                       none                         GDN0030     TiltDown 51        51   
                                                                                                   52        52   
                                                                                                   53        53   
                                                                                                   54        54   
                                                                                                   55        55   

                                                                                                      heartbeat_start_sample  \
                                                                                                                   estimated   
q_wave_algorithm b_point_algorithm               outlier_correction_algorithm participant phase                                
q-wave-reference arbol2017-isoelectric-crossings forouzanfar2018              GDN0005     Pause    0                     352   
                                                                                                   1                     808   
                                                                                                   2                    1252   
                                                                      

In [12]:
cols = [("heartbeat_start_sample", "estimated"), ("heartbeat_start_sample", "reference"), ("heartbeat_end_sample", "estimated"), ("heartbeat_end_sample", "reference"), ("q_wave_onset_sample", "estimated"), ("q_wave_onset_sample", "reference"), ("b_point_sample", "estimated"), ("b_point_sample", "reference"), ("pep_sample", "estimated"), ("pep_sample", "reference")]
results_empkins_cleaned_ms = results_empkins_norm.copy()
results_guardian_cleaned_ms = results_guardian_norm.copy()
results_empkins_cleaned_ms[cols] = results_empkins_norm[cols] * convert_hz_to_ms(sampling_frequency=fs_empkins)
results_guardian_cleaned_ms[cols] = results_guardian_norm[cols] * convert_hz_to_ms(sampling_frequency=fs_guardian)

In [13]:
results_guardian_cleaned_ms

heartbeat_id  \
                                                                                                         estimated   
q_wave_algorithm b_point_algorithm               outlier_correction_algorithm participant phase                      
q-wave-reference arbol2017-isoelectric-crossings forouzanfar2018              GDN0005     Pause    0             0   
                                                                                                   1             1   
                                                                                                   2             2   
                                                                                                   3             3   
                                                                                                   4             4   
...                                                                                                            ...   
                 stern1985                       none                         GDN0030     TiltDown 51           51   
                                                                                                   52           52   
                                                                                                   53           53   
                                                                                                   54           54   
                                                                                                   55           55   

                                                                                                                 \
                                                                                                      reference   
q_wave_algorithm b_point_algorithm               outlier_correction_algorithm participant phase                   
q-wave-reference arbol2017-isoelectric-crossings forouzanfar2018              GDN0005     Pause    0          0   
                                                                                                   1          1   
                                                                                                   2          2   
                                                                                                   3          3   
                                                                                                   4          4   
...                                                                                                         ...   
                 stern1985                       none                         GDN0030     TiltDown 51        51   
                                                                                                   52        52   
                                                                                                   53        53   
                                                                                                   54        54   
                                                                                                   55        55   

                                                                                                      heartbeat_start_sample  \
                                                                                                                   estimated   
q_wave_algorithm b_point_algorithm               outlier_correction_algorithm participant phase                                
q-wave-reference arbol2017-isoelectric-crossings forouzanfar2018              GDN0005     Pause    0                   704.0   
                                                                                                   1                  1616.0   
                                                                                                   2                  2504.0   
                                                                      

In [14]:
preprocessed_results_empkins_guardian = pd.concat([results_empkins_cleaned_ms, results_guardian_cleaned_ms])
preprocessed_results_empkins_guardian

heartbeat_id  \
                                                                                                         estimated   
q_wave_algorithm b_point_algorithm               outlier_correction_algorithm participant phase                      
q-wave-reference arbol2017-isoelectric-crossings forouzanfar2018              VP_001      Prep     0             0   
                                                                                                   1             1   
                                                                                                   2             2   
                                                                                                   3             3   
                                                                                                   4             4   
...                                                                                                            ...   
                 stern1985                       none                         GDN0030     TiltDown 51           51   
                                                                                                   52           52   
                                                                                                   53           53   
                                                                                                   54           54   
                                                                                                   55           55   

                                                                                                                 \
                                                                                                      reference   
q_wave_algorithm b_point_algorithm               outlier_correction_algorithm participant phase                   
q-wave-reference arbol2017-isoelectric-crossings forouzanfar2018              VP_001      Prep     0          0   
                                                                                                   1          1   
                                                                                                   2          2   
                                                                                                   3          3   
                                                                                                   4          4   
...                                                                                                         ...   
                 stern1985                       none                         GDN0030     TiltDown 51        51   
                                                                                                   52        52   
                                                                                                   53        53   
                                                                                                   54        54   
                                                                                                   55        55   

                                                                                                      heartbeat_start_sample  \
                                                                                                                   estimated   
q_wave_algorithm b_point_algorithm               outlier_correction_algorithm participant phase                                
q-wave-reference arbol2017-isoelectric-crossings forouzanfar2018              VP_001      Prep     0                   394.0   
                                                                                                   1                   984.0   
                                                                                                   2                  1569.0   
                                                                      

### Check whether the concatendated dataframe has the same size as the summed non-concatenated dataframes

In [15]:
print(preprocessed_results_empkins_guardian.shape[0] - (results_empkins_cleaned_ms.shape[0] + results_guardian_cleaned_ms.shape[0]))

0


### Data with or without outlier correction

In [16]:
drop_outlier_correction = True

In [17]:
input_data = preprocessed_results_empkins_guardian
if drop_outlier_correction: 
    input_data = input_data.xs(key='none', level='outlier_correction_algorithm', drop_level=True)

In [18]:
input_data

heartbeat_id  \
                                                                            estimated   
q_wave_algorithm b_point_algorithm               participant phase                      
q-wave-reference arbol2017-isoelectric-crossings VP_001      Prep     0             0   
                                                                      1             1   
                                                                      2             2   
                                                                      3             3   
                                                                      4             4   
...                                                                               ...   
                 stern1985                       GDN0030     TiltDown 51           51   
                                                                      52           52   
                                                                      53           53   
                                                                      54           54   
                                                                      55           55   

                                                                                    \
                                                                         reference   
q_wave_algorithm b_point_algorithm               participant phase                   
q-wave-reference arbol2017-isoelectric-crossings VP_001      Prep     0          0   
                                                                      1          1   
                                                                      2          2   
                                                                      3          3   
                                                                      4          4   
...                                                                            ...   
                 stern1985                       GDN0030     TiltDown 51        51   
                                                                      52        52   
                                                                      53        53   
                                                                      54        54   
                                                                      55        55   

                                                                         heartbeat_start_sample  \
                                                                                      estimated   
q_wave_algorithm b_point_algorithm               participant phase                                
q-wave-reference arbol2017-isoelectric-crossings VP_001      Prep     0                   394.0   
                                                                      1                   984.0   
                                                                      2                  1569.0   
                                                                      3                  2134.0   
                                                                      4                  2684.0   
...                                                                                         ...   
                 stern1985                       GDN0030     TiltDown 51                54058.0   
                                                                      52                55140.0   
                                                                      53                56236.0   
                                                                      54                57304.0   
                                                                      55                58334.0   

                                                                                    \
                                                                         reference   
q_wave_algorithm b_point_algorithm               participant p

In [19]:
input_data = input_data[[("heartbeat_id", "reference"), ("b_point_sample", "estimated"), ("b_point_sample", "reference"), ("rr_interval_ms", "estimated")]]
input_data.columns = input_data.columns.to_flat_index().str.join("_")
input_data

heartbeat_id_reference  \
q_wave_algorithm b_point_algorithm               participant phase                                 
q-wave-reference arbol2017-isoelectric-crossings VP_001      Prep     0                        0   
                                                                      1                        1   
                                                                      2                        2   
                                                                      3                        3   
                                                                      4                        4   
...                                                                                          ...   
                 stern1985                       GDN0030     TiltDown 51                      51   
                                                                      52                      52   
                                                                      53                      53   
                                                                      54                      54   
                                                                      55                      55   

                                                                          b_point_sample_estimated  \
q_wave_algorithm b_point_algorithm               participant phase                                   
q-wave-reference arbol2017-isoelectric-crossings VP_001      Prep     0                      275.0   
                                                                      1                      254.0   
                                                                      2                      265.0   
                                                                      3                      260.0   
                                                                      4                      259.0   
...                                                                                            ...   
                 stern1985                       GDN0030     TiltDown 51                     496.0   
                                                                      52                     436.0   
                                                                      53                     454.0   
                                                                      54                     388.0   
                                                                      55                     394.0   

                                                                          b_point_sample_reference  \
q_wave_algorithm b_point_algorithm               participant phase                                   
q-wave-reference arbol2017-isoelectric-crossings VP_001      Prep     0                      240.0   
                                                                      1                      222.0   
                                                                      2                      242.0   
                                                                      3                      248.0   
                                                                      4                      242.0   
...                                                                                            ...   
                 stern1985                       GDN0030     TiltDown 51                     498.0   
                                                                      52                     490.0   
                                                                      53                     454.0   
                                                                      54                     430.0   
                                                                      55                     444.0   

                                                                          rr_interval_ms_estimated  
q_wave_algorithm b

In [20]:
input_data.xs(key='VP_022', level='participant').max()

heartbeat_id_reference       39.0
b_point_sample_estimated    470.0
b_point_sample_reference    461.0
rr_interval_ms_estimated    952.0
dtype: float64

In [21]:
if drop_outlier_correction == False:
    input_data["algo_combis"] = input_data.index.get_level_values("b_point_algorithm") + "_" + input_data.index.get_level_values("outlier_correction_algorithm")
    input_data = input_data.reset_index().drop(columns=["b_point_algorithm", "outlier_correction_algorithm"])
input_data

heartbeat_id_reference  \
q_wave_algorithm b_point_algorithm               participant phase                                 
q-wave-reference arbol2017-isoelectric-crossings VP_001      Prep     0                        0   
                                                                      1                        1   
                                                                      2                        2   
                                                                      3                        3   
                                                                      4                        4   
...                                                                                          ...   
                 stern1985                       GDN0030     TiltDown 51                      51   
                                                                      52                      52   
                                                                      53                      53   
                                                                      54                      54   
                                                                      55                      55   

                                                                          b_point_sample_estimated  \
q_wave_algorithm b_point_algorithm               participant phase                                   
q-wave-reference arbol2017-isoelectric-crossings VP_001      Prep     0                      275.0   
                                                                      1                      254.0   
                                                                      2                      265.0   
                                                                      3                      260.0   
                                                                      4                      259.0   
...                                                                                            ...   
                 stern1985                       GDN0030     TiltDown 51                     496.0   
                                                                      52                     436.0   
                                                                      53                     454.0   
                                                                      54                     388.0   
                                                                      55                     394.0   

                                                                          b_point_sample_reference  \
q_wave_algorithm b_point_algorithm               participant phase                                   
q-wave-reference arbol2017-isoelectric-crossings VP_001      Prep     0                      240.0   
                                                                      1                      222.0   
                                                                      2                      242.0   
                                                                      3                      248.0   
                                                                      4                      242.0   
...                                                                                            ...   
                 stern1985                       GDN0030     TiltDown 51                     498.0   
                                                                      52                     490.0   
                                                                      53                     454.0   
                                                                      54                     430.0   
                                                                      55                     444.0   

                                                                          rr_interval_ms_estimated  
q_wave_algorithm b

In [30]:
columns = ""
if drop_outlier_correction:
    columns="b_point_algorithm"
else:
    columns="algo_combis"
# Pivot the DataFrame
input_data_wide = input_data.pivot_table(
    index=['participant', 'phase', 'heartbeat_id_reference', 'b_point_sample_reference', 'rr_interval_ms_estimated'],
    columns=columns,
    values='b_point_sample_estimated'
).reset_index()

# Flatten the columns
input_data_wide.columns.name = None
input_data_wide.columns = [f'{col}' if isinstance(col, str) else f'{col[1]}' for col in input_data_wide.columns]
input_data_wide

participant          phase  heartbeat_id_reference  \
0         GDN0005  HoldingBreath                       0   
1         GDN0005  HoldingBreath                       1   
2         GDN0005  HoldingBreath                       3   
3         GDN0005  HoldingBreath                       4   
4         GDN0005  HoldingBreath                       5   
...           ...            ...                     ...   
11133      VP_032           Talk                      39   
11134      VP_032           Talk                      40   
11135      VP_032           Talk                      41   
11136      VP_032           Talk                      42   
11137      VP_032           Talk                      43   

       b_point_sample_reference  rr_interval_ms_estimated  \
0                         388.0                     850.0   
1                         404.0                     778.0   
2                         376.0                     746.0   
3                         390.0                     766.0   
4                         386.0                     790.0   
...                         ...                       ...   
11133                     310.0                     619.0   
11134                     322.0                     680.0   
11135                     340.0                     719.0   
11136                     311.0                     698.0   
11137                     307.0                     689.0   

       arbol2017-isoelectric-crossings  arbol2017-second-derivative  \
0                                438.0                        398.0   
1                                340.0                        350.0   
2                                382.0                        296.0   
3                                394.0                        344.0   
4                                398.0                        312.0   
...                                ...                          ...   
11133                            335.0                        276.0   
11134                            331.0                        298.0   
11135                            317.0                        300.0   
11136                            365.0                        324.0   
11137                            342.0                        313.0   

       arbol2017-third-derivative  debski1993-second-derivative  drost2022  \
0                           394.0                         452.0      400.0   
1                           244.0                         388.0      420.0   
2                           386.0                         366.0      386.0   
3                           396.0                         376.0      396.0   
4                           388.0                         418.0      392.0   
...                           ...                           ...        ...   
11133                       324.0                         300.0      318.0   
11134                       330.0                         311.0      322.0   
11135                       348.0                         330.0      343.0   
11136                       366.0                         347.0      354.0   
11137                       356.0                         285.0      332.0   

       forounzafar2018  lozano2007-linear-regression  \
0                550.0                         412.0   
1                402.0                         404.0   
2                388.0                         366.0   
3                398.0                         372.0   
4                390.0                         378.0   
...                ...                           ...   
11133            306.0                         305.0   
11134            311.0                         321.0   
11135            309.0                         332.0   
11136            305.0                         351.0   
11137            302.0                         341.0   

       lozano2007-quadratic-regression  sherwood1990  stern1985  
0                      

In [26]:
input_data_wide.isna().any()

participant                        False
phase                              False
heartbeat_id_reference             False
b_point_sample_reference           False
rr_interval_ms_estimated           False
arbol2017-isoelectric-crossings     True
arbol2017-second-derivative        False
arbol2017-third-derivative          True
debski1993-second-derivative        True
drost2022                          False
forounzafar2018                     True
lozano2007-linear-regression       False
lozano2007-quadratic-regression    False
sherwood1990                        True
stern1985                           True
dtype: bool

In [38]:
test_amount_nan = input_data_wide[["participant", "phase", "heartbeat_id_reference", "b_point_sample_reference", "rr_interval_ms_estimated", "drost2022", "forounzafar2018", "lozano2007-quadratic-regression", "lozano2007-linear-regression", "debski1993-second-derivative"]]
test_amount_nan

participant          phase  heartbeat_id_reference  \
0         GDN0005  HoldingBreath                       0   
1         GDN0005  HoldingBreath                       1   
2         GDN0005  HoldingBreath                       3   
3         GDN0005  HoldingBreath                       4   
4         GDN0005  HoldingBreath                       5   
...           ...            ...                     ...   
11133      VP_032           Talk                      39   
11134      VP_032           Talk                      40   
11135      VP_032           Talk                      41   
11136      VP_032           Talk                      42   
11137      VP_032           Talk                      43   

       b_point_sample_reference  rr_interval_ms_estimated  drost2022  \
0                         388.0                     850.0      400.0   
1                         404.0                     778.0      420.0   
2                         376.0                     746.0      386.0   
3                         390.0                     766.0      396.0   
4                         386.0                     790.0      392.0   
...                         ...                       ...        ...   
11133                     310.0                     619.0      318.0   
11134                     322.0                     680.0      322.0   
11135                     340.0                     719.0      343.0   
11136                     311.0                     698.0      354.0   
11137                     307.0                     689.0      332.0   

       forounzafar2018  lozano2007-quadratic-regression  \
0                550.0                            384.0   
1                402.0                            384.0   
2                388.0                            348.0   
3                398.0                            348.0   
4                390.0                            354.0   
...                ...                              ...   
11133            306.0                            294.0   
11134            311.0                            303.0   
11135            309.0                            322.0   
11136            305.0                            337.0   
11137            302.0                            329.0   

       lozano2007-linear-regression  debski1993-second-derivative  
0                             412.0                         452.0  
1                             404.0                         388.0  
2                             366.0                         366.0  
3                             372.0                         376.0  
4                             378.0                         418.0  
...                             ...                           ...  
11133                         305.0                         300.0  
11134                         321.0                         311.0  
11135                         332.0                         330.0  
11136                         351.0                         347.0  
11137                         341.0                         285.0  

[11138 rows x 10 columns]

In [39]:
test_amount_nan.isna().any()

participant                        False
phase                              False
heartbeat_id_reference             False
b_point_sample_reference           False
rr_interval_ms_estimated           False
drost2022                          False
forounzafar2018                     True
lozano2007-quadratic-regression    False
lozano2007-linear-regression       False
debski1993-second-derivative        True
dtype: bool

In [40]:
test_amount_nan.dropna()

participant          phase  heartbeat_id_reference  \
0         GDN0005  HoldingBreath                       0   
1         GDN0005  HoldingBreath                       1   
2         GDN0005  HoldingBreath                       3   
3         GDN0005  HoldingBreath                       4   
4         GDN0005  HoldingBreath                       5   
...           ...            ...                     ...   
11133      VP_032           Talk                      39   
11134      VP_032           Talk                      40   
11135      VP_032           Talk                      41   
11136      VP_032           Talk                      42   
11137      VP_032           Talk                      43   

       b_point_sample_reference  rr_interval_ms_estimated  drost2022  \
0                         388.0                     850.0      400.0   
1                         404.0                     778.0      420.0   
2                         376.0                     746.0      386.0   
3                         390.0                     766.0      396.0   
4                         386.0                     790.0      392.0   
...                         ...                       ...        ...   
11133                     310.0                     619.0      318.0   
11134                     322.0                     680.0      322.0   
11135                     340.0                     719.0      343.0   
11136                     311.0                     698.0      354.0   
11137                     307.0                     689.0      332.0   

       forounzafar2018  lozano2007-quadratic-regression  \
0                550.0                            384.0   
1                402.0                            384.0   
2                388.0                            348.0   
3                398.0                            348.0   
4                390.0                            354.0   
...                ...                              ...   
11133            306.0                            294.0   
11134            311.0                            303.0   
11135            309.0                            322.0   
11136            305.0                            337.0   
11137            302.0                            329.0   

       lozano2007-linear-regression  debski1993-second-derivative  
0                             412.0                         452.0  
1                             404.0                         388.0  
2                             366.0                         366.0  
3                             372.0                         376.0  
4                             378.0                         418.0  
...                             ...                           ...  
11133                         305.0                         300.0  
11134                         321.0                         311.0  
11135                         332.0                         330.0  
11136                         351.0                         347.0  
11137                         341.0                         285.0  

[10569 rows x 10 columns]

In [27]:
input_data_wide = input_data_wide.dropna()
input_data_wide

participant          phase  heartbeat_id_reference  \
0         GDN0005  HoldingBreath                       0   
1         GDN0005  HoldingBreath                       1   
2         GDN0005  HoldingBreath                       3   
3         GDN0005  HoldingBreath                       4   
4         GDN0005  HoldingBreath                       5   
...           ...            ...                     ...   
11133      VP_032           Talk                      39   
11134      VP_032           Talk                      40   
11135      VP_032           Talk                      41   
11136      VP_032           Talk                      42   
11137      VP_032           Talk                      43   

       b_point_sample_reference  rr_interval_ms_estimated  \
0                         388.0                     850.0   
1                         404.0                     778.0   
2                         376.0                     746.0   
3                         390.0                     766.0   
4                         386.0                     790.0   
...                         ...                       ...   
11133                     310.0                     619.0   
11134                     322.0                     680.0   
11135                     340.0                     719.0   
11136                     311.0                     698.0   
11137                     307.0                     689.0   

       arbol2017-isoelectric-crossings  arbol2017-second-derivative  \
0                                438.0                        398.0   
1                                340.0                        350.0   
2                                382.0                        296.0   
3                                394.0                        344.0   
4                                398.0                        312.0   
...                                ...                          ...   
11133                            335.0                        276.0   
11134                            331.0                        298.0   
11135                            317.0                        300.0   
11136                            365.0                        324.0   
11137                            342.0                        313.0   

       arbol2017-third-derivative  debski1993-second-derivative  drost2022  \
0                           394.0                         452.0      400.0   
1                           244.0                         388.0      420.0   
2                           386.0                         366.0      386.0   
3                           396.0                         376.0      396.0   
4                           388.0                         418.0      392.0   
...                           ...                           ...        ...   
11133                       324.0                         300.0      318.0   
11134                       330.0                         311.0      322.0   
11135                       348.0                         330.0      343.0   
11136                       366.0                         347.0      354.0   
11137                       356.0                         285.0      332.0   

       forounzafar2018  lozano2007-linear-regression  \
0                550.0                         412.0   
1                402.0                         404.0   
2                388.0                         366.0   
3                398.0                         372.0   
4                390.0                         378.0   
...                ...                           ...   
11133            306.0                         305.0   
11134            311.0                         321.0   
11135            309.0                         332.0   
11136            305.0                         351.0   
11137            302.0                         341.0   

       lozano2007-quadratic-regression  sherwood1990  stern1985  
0                      

In [28]:
print(f"Save results: {save_results}")

Save results: True


In [29]:
if save_results:
    input_data_wide.to_csv(result_path.joinpath("data/train_data_rr_interval_no_outlier_correction.csv"))